In [1]:
import pandas as pd
from sklearn.metrics import make_scorer, balanced_accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from hyperopt import hp
import numpy as np
from hyperopt import Trials, tpe, fmin
import warnings
from imblearn.under_sampling import RandomUnderSampler
from sklearn.svm import SVC
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from nbimporter import NotebookLoader

warnings.filterwarnings("ignore")

In [2]:
ma = pd.read_csv("../16 Mycoplosma/ma.csv")
map = pd.read_csv("../16 Mycoplosma/map.csv")
marth = pd.read_csv("../16 Mycoplosma/marth.csv")
mcap = pd.read_csv("../16 Mycoplosma/mcap.csv")
mcon = pd.read_csv("../16 Mycoplosma/mcon.csv")
mcro = pd.read_csv("../16 Mycoplosma/mcro.csv")
mgal = pd.read_csv("../16 Mycoplosma/mgal.csv")
mhom = pd.read_csv("../16 Mycoplosma/mhom.csv")
mhyoJ = pd.read_csv("../16 Mycoplosma/mhyoJ.csv")
mhyo232 = pd.read_csv("../16 Mycoplosma/mhyo232.csv")
mhyo7448 = pd.read_csv("../16 Mycoplosma/mhyo7448.csv")
mm163K = pd.read_csv("../16 Mycoplosma/mm163K.csv")
mms = pd.read_csv("../16 Mycoplosma/mms.csv")
mpHF = pd.read_csv("../16 Mycoplosma/mpHF.csv")
mpM = pd.read_csv("../16 Mycoplosma/mpM.csv")
ms53 = pd.read_csv("../16 Mycoplosma/ms53.csv")

In [3]:
def generate_train_test_dataset(index):
    dataset_names = [ma, map, marth, mcap, mcon, mcro, mgal, mhom, mhyoJ, mhyo232, mhyo7448, mm163K, mms, mpHF, mpM, ms53]
    merged_train = dataset_names[:index]+dataset_names[index+1:]
    train_data = pd.concat(merged_train)
    test_data = dataset_names[index]
    return train_data,test_data

In [4]:
def handle_dataset(dataset):
    columns_to_drop = ['identity','name', 'Amino acid', 'protein_sequence', 'essentiality']
    Xs = dataset.drop(columns = columns_to_drop)
    y = dataset['essentiality']
    return Xs,y

In [5]:
rus = RandomUnderSampler(sampling_strategy='auto', random_state=42)

In [6]:
from sklearn.linear_model import LassoCV

lassoCV = LassoCV(cv=20,random_state=10)
all_feature_names = ["GC_Content","CAI","A","R","N","D","C","Q","E","G","H","I","L","K","M","F","P","S","T","W","Y","V","nSE2","nSE3","nGE2","nGE3"]

In [7]:
space = {
    'C': hp.loguniform('C', -5, 2),  # 正则化参数 C
    'degree': hp.quniform('degree', 2, 5, 1),  # 多项式核的次数
    'gamma': hp.choice('gamma', ['auto', 'scale', hp.loguniform('gamma_val', -5, 2)])  # 核函数的系数
}

In [8]:

from sklearn.metrics import accuracy_score

def svm_ac_cv(params):
    model = SVC(C = params["C"],kernel ="linear", degree = int(params["degree"]), gamma = params["gamma"])
    scorer = make_scorer(accuracy_score)
    score = -cross_val_score(model, X_resampled, y_resampled, cv=5,scoring=scorer ).mean()
    return score


In [9]:
from joblib import load
from sklearn.feature_selection import RFE

scores = []
score = 0
models = []
model_svm_fs = load("../../model/SVM_model.joblib")
for i in range(0,15):
    train_data, test_data = generate_train_test_dataset(i)
    Xs_train,y_train = handle_dataset(train_data)
    Xs_test,y_test = handle_dataset(test_data)
    rus = RandomUnderSampler(sampling_strategy='auto', random_state=42)
    X_resampled, y_resampled = rus.fit_resample(Xs_train, y_train)
    rfe = RFE(model_svm_fs, n_features_to_select=20)
    rfe.fit(X_resampled, y_resampled)
    X_resampled = rfe.transform(X_resampled)  # For training data
    Xs_test = rfe.transform(Xs_test)  # For testing data
    lassoCV.fit(X_resampled,y_resampled)
    selected_features = [feature for feature, coef in zip(all_feature_names, lassoCV.coef_) if coef !=0]
    trials = Trials()
    selected_feature_indices = [all_feature_names.index(feature) for feature in selected_features]
    X_resampled = X_resampled[:, selected_feature_indices]
    Xs_test = Xs_test[:,selected_feature_indices]
    params=fmin(fn=svm_ac_cv, # function to optimize
              space=space, 
              algo=tpe.suggest, # optimization algorithm, hyperotp will select its parameters automatically
              max_evals=10, # maximum number of iterations
              trials=trials, # logging
              rstate=np.random.default_rng(42) # fixing random state for the reproducibility
    )
    model = SVC(C = params["C"],kernel ="linear", degree = int(params["degree"]), gamma = params["gamma"])
    models.append(model)
    model.fit(X_resampled,y_resampled)
    tpe_test_score=accuracy_score(y_test, model.predict(Xs_test))
    scores.append(tpe_test_score)
    print(tpe_test_score)
    if score < tpe_test_score:
        score = tpe_test_score
        best_model = model

100%|██████████| 10/10 [00:00<00:00, 10.87trial/s, best loss: -0.850728554641598]
0.8885941644562334
100%|██████████| 10/10 [00:00<00:00, 11.31trial/s, best loss: -0.8692307692307691]
0.855072463768116
100%|██████████| 10/10 [00:00<00:00, 10.29trial/s, best loss: -0.8540448287632267]
0.7327044025157232
100%|██████████| 10/10 [00:00<00:00, 10.22trial/s, best loss: -0.8520271353026013]
0.9224489795918367
100%|██████████| 10/10 [00:00<00:00, 10.83trial/s, best loss: -0.8498476223333908]
0.8957055214723927
100%|██████████| 10/10 [00:00<00:00, 11.00trial/s, best loss: -0.8508909479686387]
0.903954802259887
100%|██████████| 10/10 [00:00<00:00, 10.63trial/s, best loss: -0.8561339600470035]
0.7627118644067796
100%|██████████| 10/10 [00:00<00:00, 11.05trial/s, best loss: -0.8531021792881376]
0.9258064516129032
100%|██████████| 10/10 [00:00<00:00, 10.75trial/s, best loss: -0.8743201729434519]
0.6791907514450867
100%|██████████| 10/10 [00:00<00:00, 11.26trial/s, best loss: -0.842596077955194]
0.8

In [10]:
print(score)

0.9504132231404959


In [11]:
train_data, test_data = generate_train_test_dataset(12)
Xs_train,y_train = handle_dataset(train_data)
X_resampled, y_resampled = rus.fit_resample(Xs_train, y_train)
rfe = RFE(model_svm_fs, n_features_to_select=20)
rfe.fit(X_resampled, y_resampled)
X_resampled = rfe.transform(X_resampled)
lassoCV.fit(X_resampled,y_resampled)
selected_feature_indices = rfe.get_support(indices=True)
print(selected_feature_indices)
selected_feature_names = [all_feature_names[index-1] for index in selected_feature_indices]
selected_features = [feature for feature, coef in zip(selected_feature_names, lassoCV.coef_) if coef !=0]
print("Selected feature RFE:", selected_feature_names)
print("Selected feature Lasso:", selected_features)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 18 19 21 22]
Selected feature RFE: ['GC_Content', 'CAI', 'A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'S', 'T', 'Y', 'V']
Selected feature Lasso: ['GC_Content', 'CAI', 'A', 'R', 'N', 'D', 'E', 'G', 'H', 'K', 'M', 'F', 'S', 'T', 'Y', 'V']


In [12]:
from joblib import dump

dump(best_model, '../../model/SVM_model_FSS.joblib')

['../../model/SVM_model_FSS.joblib']

In [12]:
train_data, test_data = generate_train_test_dataset(12)
Xs_train,y_train = handle_dataset(train_data)
Xs_test,y_test = handle_dataset(test_data)
rus = RandomUnderSampler(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = rus.fit_resample(Xs_train, y_train)
lassoCV.fit(X_resampled,y_resampled)
selected_features = [feature for feature, coef in zip(all_feature_names, lassoCV.coef_) if coef !=0]
print(selected_features)

['GC_Content', 'CAI', 'A', 'N', 'D', 'H', 'M', 'nSE3', 'nGE3']
